# ETL  || Planning -> Develope -> Manage & Automate

##### Source (From Where We Extract) -> We Transform (basis Our problem) -> Finally, Loading transformed data into our Target

#### Extracting Data By Running Daily This Python API Script of NIfty50 Stock !

In [30]:
import pandas as pd
from bsedata.bse import BSE  ## if not installed use: pip install bsedata
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

github_excel_url = "https://raw.githubusercontent.com/jangid6/Stock-ETL-Project/main/Equity.xlsx"
engine = 'openpyxl'  ## if not installed use pip install openpyxl
EquityDF = pd.read_excel(github_excel_url, engine = engine)
EquityDF.head(n=2) # Get the list of stocks in Nifty50

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas


In [31]:
len(EquityDF) # Just 50 stocks which are part of nifty50 index

4244

##### CALLING BSE API for fetching Stocks Data for eg: Price, Code, Updated Date, Open Price, Close Price, Mrkt Cap

In [32]:
listOf_Nifty50_StockIDs = [ 
    "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
    "UPL", "ULTRACEMCO", "WIPRO"
]

## We Want to filter EquityDF by looking up 'Security Id' col values exist in listOf_Nifty50_StockIDs 
EquityDF['Security Code'] = EquityDF['Security Code'].astype(str) ##coverting into str as getQuote() accepts str input

nifty50_OnlyDF= EquityDF[EquityDF['Security Id'].isin(listOf_Nifty50_StockIDs)].reset_index(drop=True)
nifty50_OnlyDF.columns = nifty50_OnlyDF.columns.str.replace(' ', '')
nifty50_OnlyDF.head()

,SecurityCode,IssuerName,SecurityId,SecurityName,Status,Group,FaceValue,ISINNo,Industry,Instrument,SectorName,IndustryNewName,IgroupName,ISubgroupName
0,500034,Bajaj Finance Limited,BAJFINANCE,Bajaj Finance Limited,Active,A,2.0,INE296A01024,Non Banking Financial Company (NBFC),Equity,Financial Services,Financial Services,Finance,Non Banking Financial Company (NBFC)
1,500087,"Cipla Ltd.,",CIPLA,CIPLA LTD.,Active,A,2.0,INE059A01026,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals
2,500112,"State Bank Of India,",SBIN,STATE BANK OF INDIA,Active,A,1.0,INE062A01020,Public Sector Bank,Equity,Financial Services,Financial Services,Banks,Public Sector Bank
3,500114,Titan Company Limited,TITAN,Titan Company Limited,Active,A,1.0,INE280A01028,Gems- Jewellery And Watches,Equity,Consumer Discretionary,Consumer Durables,Consumer Durables,"Gems, Jewellery And Watches"
4,500124,"Dr. Reddy's Laboratories Ltd.,",DRREDDY,DR.REDDY'S LABORATORIES LTD.,Active,A,5.0,INE089A01023,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals


In [33]:
nifty50_OnlyDF['SecurityCode'].values  

array(['500034', '500087', '500112', '500114', '500124', '500180',
       '500182', '500209', '500228', '500247', '500300', '500312',
       '500325', '500440', '500470', '500510', '500520', '500547',
       '500570', '500696', '500790', '500800', '500820', '500825',
       '500875', '505200', '507685', '508869', '512070', '512599',
       '524715', '532174', '532187', '532215', '532281', '532454',
       '532488', '532500', '532538', '532540', '532555', '532755',
       '532898', '532921', '532977', '532978', '533278', '540005',
       '540719', '540777'], dtype=object)

In [34]:
bseObject = BSE(update_codes=True)  ##creating Bse Lib Object

listof_stockDicts = []
sqcode_ListNifty50 = nifty50_OnlyDF['SecurityCode'].values  
for sqCode in sqcode_ListNifty50:
    try:
        stockDict = bseObject.getQuote(sqCode) #key-value pair
        stockDict.pop("buy", None)
        stockDict.pop("sell", None)
        listof_stockDicts.append(stockDict)
        time.sleep(0.5)
    except IndexError:
        print(f"IndexError for {sqCode}: Data not available")

nifty50DailyTable = pd.DataFrame(listof_stockDicts)
nifty50DailyTable.head()

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,7129.75,3.60,0.05,29 Nov 23 | 04:00 PM,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,7126.15,7130.05,7169.90,7115.00,8190.00,5487.25,7133.21,8.94 Cr.,0.13 Lakh,0.52 Lakh,"4,40,635.00 Cr.","1,93,879.40 Cr."
1,CIPLA LTD.,1201.90,9.85,0.83,29 Nov 23 | 04:00 PM,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,1192.05,1194.55,1205.90,1191.75,1283.00,852.00,1201.07,8.66 Cr.,0.72 Lakh,0.81 Lakh,"97,032.44 Cr.","64,041.41 Cr."
2,STATE BANK OF INDIA,568.50,3.95,0.70,29 Nov 23 | 04:00 PM,SBIN,500112,A / S&P BSE SENSEX,1.00,Financial Services,564.55,567.30,569.00,565.05,629.65,499.35,567.52,36.83 Cr.,6.49 Lakh,5.75 Lakh,"5,07,364.19 Cr.","2,18,166.60 Cr."
3,Titan Company Limited,3427.25,-14.80,-0.43,29 Nov 23 | 04:00 PM,TITAN,500114,A / S&P BSE SENSEX,1.00,Consumer Durables,3442.05,3455.00,3470.00,3423.60,3470.00,2268.90,3438.65,3.34 Cr.,0.10 Lakh,0.28 Lakh,"3,04,266.51 Cr.","1,43,005.26 Cr."
4,DR.REDDY'S LABORATORIES LTD.,5715.55,44.30,0.78,29 Nov 23 | 04:00 PM,DRREDDY,500124,A / S&P BSE 100,5.00,Healthcare,5671.25,5700.00,5742.45,5654.55,5986.20,4176.85,5704.21,3.84 Cr.,0.07 Lakh,0.16 Lakh,"95,329.11 Cr.","69,590.25 Cr."


#####  Data Pre-Processing (Cleaning and Transformation)  for Microsoft SQL Server Database

In [35]:
nifty50DailyTable.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50DailyTable.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)
# Convert 'updatedOn' column to datetime and extract date
nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check if there are any invalid or missing date values
if pd.isna(nifty50DailyTable['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50DailyTable['updatedOn'] = pd.to_datetime(nifty50DailyTable['updatedOn'].dt.date)

if 'totalTradedValueCr' not in nifty50DailyTable.columns:
   # Assuming nifty50DailyTable is your DataFrame
    nifty50DailyTable['totalTradedValueCr'] = pd.to_numeric(nifty50DailyTable['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['totalTradedQuantityLakh'] = pd.to_numeric(nifty50DailyTable['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50DailyTable['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTable['marketCapFullCr'] = pd.to_numeric(nifty50DailyTable['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTable['marketCapFreeFloatCr'] = pd.to_numeric(nifty50DailyTable['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'

    # Drop original columns
    nifty50DailyTable.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

nifty50DailyTable.head(n=2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,7129.75,3.60,0.05,2023-11-29,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,7126.15,7130.05,7169.90,7115.00,8190.00,5487.25,7133.21,8.94,0.13,0.52,440635.00,193879.40
1,CIPLA LTD.,1201.90,9.85,0.83,2023-11-29,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,1192.05,1194.55,1205.90,1191.75,1283.00,852.00,1201.07,8.66,0.72,0.81,97032.44,64041.41


#### Loading data to Microsoft SQL Server Database Using pyodbc connection and Sqlalchemy Engine

In [36]:
import pyodbc ##for SQL Queries: pip install pyodbc
from sqlalchemy import create_engine, inspect ##prebuilt toolkit to work with SQL database
from sqlalchemy.exc import SQLAlchemyError
import sqlalchemy as sa

server = 'localhost'
database = 'nifty50'
username = 'sa'
password = 'jangid6'
driver = 'ODBC Driver 17 for SQL Server'

# database -> Driver -> to crate an engine -> so you can perfomr operations in the database

# Azure SQL Database connection string
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Create an SQLAlchemy engine
engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}")

def create_connection(conn_str):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    return conn, cursor

try:
    # Try to connect to the SQL Server using the engine
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
    conn,  cursor = create_connection(conn_str)
    inspector = inspect(engine)
    nifty50_table_name = 'nifty50_dailydata'
    if not inspector.has_table(nifty50_table_name):
        nifty50_table_schema = f'''
        CREATE TABLE {nifty50_table_name} (
            companyName NVARCHAR(MAX),
            currentValue FLOAT,
            change FLOAT,
            pChange FLOAT,
            updatedOn DATE,
            securityID NVARCHAR(MAX),
            scripCode NVARCHAR(MAX),
            sharegroup NVARCHAR(MAX),
            faceValue FLOAT,
            industry NVARCHAR(MAX),
            previousClose FLOAT,
            previousOpen FLOAT,
            dayHigh FLOAT,
            dayLow FLOAT,
            fiftytwoweekHigh FLOAT,
            fiftytwoweekLow FLOAT,
            weightedAvgPrice FLOAT,
            totalTradedQuantityLakh FLOAT,
            totalTradedValueCr FLOAT,
            twoWeekAvgQuantityLakh FLOAT,
            marketCapFullCr FLOAT,
            marketCapFreeFloatCr FLOAT
        );
        '''
        # Execute the schema to create the table
        cursor.execute(nifty50_table_schema)
        conn.commit()
        conn.close()

    with engine.begin() as engineConn:
        sql_max_updatedOn = pd.read_sql_query(sa.text(f'SELECT MAX(updatedOn) FROM {nifty50_table_name}'), engineConn).iloc[0, 0]
        print(sql_max_updatedOn)
        df_max_updatedOn = nifty50DailyTable['updatedOn'].max()
        print(df_max_updatedOn)
        if (pd.isnull(sql_max_updatedOn)) and (not pd.isnull(df_max_updatedOn)):
            nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
            print("Daily Data didn't exist, but now inserted successfully.")
        else:
            if (df_max_updatedOn > pd.Timestamp(sql_max_updatedOn)):
                nifty50DailyTable.to_sql(nifty50_table_name, engine, index=False, if_exists='append', method='multi')
                print("Data appended successfully.")
            else:
                print("No new data to append.")
    
    company_table_name = 'nifty50_companydata'
    if not inspector.has_table(company_table_name):
        # Define the table schema based on the 'Equity' DataFrame columns
        company_table_schema = f'''
        CREATE TABLE {company_table_name} (
            securityCode NVARCHAR(MAX),
            issuerName NVARCHAR(MAX),
            securityId NVARCHAR(MAX),
            securityName NVARCHAR(MAX),
            status NVARCHAR(MAX),
            CompanyGroup NVARCHAR(MAX),
            faceValue FLOAT,
            isinNo NVARCHAR(MAX),
            industry NVARCHAR(MAX),
            instrument NVARCHAR(MAX),
            sectorName NVARCHAR(MAX),
            industryNewName NVARCHAR(MAX),
            igroupName NVARCHAR(MAX),
            iSubgroupName NVARCHAR(MAX)
        );
        '''

        # Execute the schema to create the 'company' table
        conn , cursor = create_connection(conn_str)
        cursor.execute(company_table_schema)
        # Commit the changes and close the connection
        conn.commit()
        conn.close()
        nifty50_OnlyDF.rename(columns={'Group': 'CompanyGroup'}, inplace=True)
        nifty50_OnlyDF.to_sql(company_table_name, engine, index=False, if_exists='append', method='multi')
    else:
        print("company Table already exist, hence skipping")

except SQLAlchemyError as e:
    print(f"Error connecting to SQL Server: {e}")


Connection successful!
2023-11-28
2023-11-29 00:00:00


Data appended successfully.
company Table already exist, hence skipping
